# 14章 時系列の手法を学ぼう 1

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_air_passengers.csv')
display(df)
display(df.info())

In [ ]:
time = pd.to_datetime(df['ds'])
sns.lineplot(x=time, y=df['y'])

In [ ]:
from prophet import Prophet
model = Prophet()

In [ ]:
model.fit(df)

In [ ]:
# 予測
future = model.make_future_dataframe(periods=36, freq='MS')
display(future)

In [ ]:
forecast = model.predict(future)
display(forecast)

In [ ]:
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]

In [ ]:
fig_forecast = model.plot(forecast)

In [ ]:
cutoffs = pd.to_datetime(["1954-12-01", "1955-12-01", "1956-12-01", "1957-12-01"])

In [ ]:
from prophet.diagnostics import cross_validation

In [ ]:
df_cv = cross_validation(model, horizon = '1096 days', cutoffs=cutoffs)

In [ ]:
display(df_cv)

In [ ]:
from prophet.diagnostics import performance_metrics

In [ ]:
df_p = performance_metrics(df_cv)
display(len(df_p))
display(df_p)

In [ ]:
df_p = performance_metrics(df_cv, monthly=True)
display(len(df_p))
display(df_p)

In [ ]:
sns.lineplot(x="horizon", y="mse", data=df_p)

In [ ]:
sns.lineplot(x='horizon', y='coverage', data=df_p)


# 15章: 前章の続き

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_air_passengers.csv')
display(df)

In [ ]:
from prophet import Prophet
# 周期性変動が徐々に拡大するモデル
model = Prophet(seasonality_mode="multiplicative")

In [ ]:
model.fit(df)

In [ ]:
future: pd.DataFrame = model.make_future_dataframe(periods=36, freq="MS")
display(future)

In [ ]:
forecast = model.predict(future)

In [ ]:
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]

In [ ]:
fig_forecast = model.plot(forecast)

In [ ]:
cutoffs = pd.to_datetime(["1954-12-01", "1955-12-01", "1956-12-01", "1957-12-01"])

In [ ]:
from prophet.diagnostics import cross_validation
df_cv = cross_validation(model, horizon = '1096 days', cutoffs=cutoffs)


In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv, monthly=True)
df_p.head()

In [ ]:
sns.lineplot(x='horizon', y='mse', data=df_p).set(ylim=(0, 5000))

In [ ]:
sns.lineplot(x='horizon', y='coverage', data=df_p)

## 補足1: 不確実性区間の幅の調整

In [ ]:
model = Prophet(seasonality_mode="multiplicative", interval_width=0.95)
model.fit(df)
future = model.make_future_dataframe(periods=36, freq="MS")
forecast = model.predict(future)
fig_forecast = model.plot(forecast)

In [ ]:
forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]

In [ ]:
df_cv = cross_validation(model, horizon = '1096 days', cutoffs=cutoffs)
df_p = performance_metrics(df_cv, monthly=True)
sns.lineplot(x='horizon', y="coverage", data=df_p)

# 補足2: 予測モデルの構成要素の確認

In [ ]:
fig_components = model.plot_components(forecast)

# 演習1
以下のデータはWikipediaページビューデータです。１年先までの時系列予測と評価をおこなってください。


```
data = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
```



In [ ]:
# 演習1
from prophet import Prophet
import prophet.plot as plt
import pandas as pd
import seaborn as sns

data = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
display(data.head())
display(data.info())
time = pd.to_datetime(df['ds'])
sns.lineplot(x=time, y=df['y'])


In [ ]:

df = data[['ds','y']]
m = Prophet()
m.fit(df)
# freq=Dが規定なので365日分の予測対象を追加している
future = m.make_future_dataframe(periods=365)
display(future)
forecast = m.predict(future)
display(type(forecast))
display(forecast.head())

fig = m.plot(forecast)

In [ ]:
from prophet.diagnostics import cross_validation

cutoffs = pd.to_datetime(['2011-12-31', '2012-12-31', '2013-12-31', '2014-12-31'])
df_cv = cross_validation(m, horizon="365 days", cutoffs=cutoffs, period="180 days")
df_cv


In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p

In [ ]:
sns.lineplot(x='horizon', y='mse', data=df_p)

In [ ]:
sns.lineplot(x='horizon', y='coverage', data=df_p)